In [102]:
import miditoolkit
import numpy as np
import random

In [139]:
class NoiseModule:
    def __init__(self, mido_obj,target=0):
        self.mido_obj = mido_obj
        if target==0:
            self.addNoiseToMelody()
        else:
            self.addNoiseToFull()

    def addNoiseToMelody(self):
        # TODO : decide what all noise to add
        pass

    def addNoiseToFull(self):
        # TODO : decide what all noise to add
        self.addVelocityNoise(0,0.3,0.2)
        self.addLengthNoise(0,0.3)
        self.addPitchNoise(2,0.3)
        self.addExtraNotes(0,0.2)
        self.addExtraNotes(1,0.2)

    def addVelocityNoise(self,channel,contamination,noise_perc):
        length = len(self.mido_obj.instruments[channel].notes)
        idx = random.sample(range(0,length),int(length*contamination))
        for i in idx:
            vel = self.mido_obj.instruments[channel].notes[i].velocity
            max_del_v = int(vel*noise_perc)
            del_v = random.randint(-max_del_v,max_del_v+1)
            self.mido_obj.instruments[channel].notes[i].velocity = min(vel+del_v,127)

    def addPitchNoise(self,channel,contamination,del_change=8):
        change = random.randint(-del_change,del_change+1)
        length = len(self.mido_obj.instruments[channel].notes)
        idx = random.sample(range(0,length),int(length*contamination))
        for i in idx:
            pitch = self.mido_obj.instruments[channel].notes[i].pitch
            self.mido_obj.instruments[channel].notes[i].velocity = min(pitch+change,127)

    def addExtraNotes(self,channel,contamination,thresh=0.2):
        if channel == 0:
            extra_notes = []
            for i in range(1,len(self.mido_obj.instruments[channel].notes)):
                if random.random() < contamination:
                    try:
                        prev_end = self.mido_obj.instruments[channel].notes[i-1].end
                        cur_start = self.mido_obj.instruments[channel].notes[i].start
                        length = int((cur_start-prev_end)*thresh)
                        startnoise = random.randint(0,length)
                        endnoise = random.randint(0,length)
                        starttime = prev_end+startnoise
                        endtime = cur_start-endnoise
                        prev_pitch = self.mido_obj.instruments[channel].notes[i-1].pitch
                        cur_pitch = self.mido_obj.instruments[channel].notes[i].pitch
                        pitch = random.randint(min(prev_pitch,cur_pitch),max(prev_pitch,cur_pitch))
                        #pitch = min(68,pitch)
                        prev_vel = self.mido_obj.instruments[channel].notes[i-1].velocity
                        cur_vel = self.mido_obj.instruments[channel].notes[i].velocity
                        vel = random.randint(min(prev_vel,cur_vel),max(prev_vel,cur_vel))
                        #vel = min(120,vel)
                        extra_notes.append(miditoolkit.Note(vel,pitch,starttime,endtime))
                    except:
                        pass
            extra_notes = extra_notes[1:]
            self.mido_obj.instruments[channel].notes.extend(extra_notes)
            self.mido_obj.instruments[channel].notes.sort(key=lambda x: x.start)
        else:
            extra_notes = []
            for i in range(1,len(self.mido_obj.instruments[channel].notes) -1):
                if random.random() < contamination:
                    try:
                        cur_start = self.mido_obj.instruments[channel].notes[i].start
                        cur_end = self.mido_obj.instruments[channel].notes[i].end
                        origLength = cur_end-cur_start
                        noiseDisp = int(origLength*thresh)
                        startnoise = random.randint(-noiseDisp,noiseDisp)
                        endnoise = random.randint(-noiseDisp,noiseDisp)
                        starttime = max(0,cur_start+startnoise)
                        endtime = cur_start+endnoise
                        prev_pitch = self.mido_obj.instruments[channel].notes[i-1].pitch
                        cur_pitch = self.mido_obj.instruments[channel].notes[i].pitch
                        next_pitch = self.mido_obj.instruments[channel].notes[i+1].pitch
                        pitch = random.randint(min(prev_pitch,cur_pitch,next_pitch),max(prev_pitch,cur_pitch,next_pitch))
                        #pitch = min(68,pitch)
                        prev_vel = self.mido_obj.instruments[channel].notes[i-1].velocity
                        cur_vel = self.mido_obj.instruments[channel].notes[i].velocity
                        next_vel = self.mido_obj.instruments[channel].notes[i+1].velocity
                        vel = random.randint(min(prev_vel,cur_vel,next_vel),max(prev_vel,cur_vel,next_vel))
                        #vel = min(120,vel)
                        extra_notes.append(miditoolkit.Note(vel,pitch,starttime,endtime))
                    except:
                        pass
            extra_notes = extra_notes[1:]
            self.mido_obj.instruments[channel].notes.extend(extra_notes)
            self.mido_obj.instruments[channel].notes.sort(key=lambda x: x.end)


    def addLengthNoise(self,channel,contamination,thresh=0.1):
        if channel != 0:
            for i in range(len(self.mido_obj.instruments[channel].notes)):
                if random.random() < contamination:
                    start = self.mido_obj.instruments[channel].notes[i].start
                    end = self.mido_obj.instruments[channel].notes[i].end
                    length = int((end-start)*thresh)
                    startnoise = random.randint(-length,length)
                    endnoise = random.randint(-length,length)
                    self.mido_obj.instruments[channel].notes[i].start = start+startnoise
                    self.mido_obj.instruments[channel].notes[i].end = end+endnoise
        else:
            for i in range(1,len(self.mido_obj.instruments[channel].notes)):
                if random.random() < contamination:
                    prev_start = self.mido_obj.instruments[channel].notes[i-1].start
                    prev_end = self.mido_obj.instruments[channel].notes[i-1].end
                    cur_start = self.mido_obj.instruments[channel].notes[i].start
                    cur_end = self.mido_obj.instruments[channel].notes[i].end
                    length = int((cur_start-prev_end)*thresh)
                    startnoise = random.randint(-length,length)
                    endnoise = random.randint(-length,length)
                    self.mido_obj.instruments[channel].notes[i].start = max(prev_end+startnoise,prev_start)
                    self.mido_obj.instruments[channel].notes[i].end = min(cur_start+endnoise,cur_end)

In [140]:
fname = "POP909_001_001.mid"
mido_obj = miditoolkit.midi.parser.MidiFile(fname)
output = NoiseModule(mido_obj,1).mido_obj
output_fname = fname.replace(".mid","_noise.mid")
output.dump(output_fname)

POP909_001_001_noise.mid
